Imports + env info (GRU4Rec baseline)

In [1]:
# [CELL 09-00] Imports + env info (GRU4Rec baseline)

import os
import json
import math
import time
import random
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

print("[09-00] Imports OK")
print("[09-00] torch:", torch.__version__)
print("[09-00] pandas:", pd.__version__)
print("[09-00] numpy:", np.__version__)


[09-00] Imports OK
[09-00] torch: 2.9.1+cpu
[09-00] pandas: 2.3.3
[09-00] numpy: 2.4.0


Locate repo root + fixed upstream run tags + load protocol/config artifacts

In [2]:
# [CELL 09-01] Locate repo root + fixed upstream run tags + load protocol/config artifacts

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "PROJECT_STATE.md").exists() and (p / "meta.json").exists():
            return p
    for p in [start, *start.parents]:
        if (p / "PROJECT_STATE.md").exists():
            return p
    raise FileNotFoundError("Could not locate repo root (expected PROJECT_STATE.md).")

REPO_ROOT = find_repo_root(Path.cwd().resolve())
print("[09-01] REPO_ROOT:", REPO_ROOT)

RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
print("[09-01] RUN_TAG:", RUN_TAG)

# Fixed upstream run tags (do NOT change)
TARGET_TAG = "20251229_163357"
SOURCE_TAG = "20251229_232834"

def load_json(path: Path) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

cfg_path_repo = REPO_ROOT / "data/processed/supervised" / f"dataloader_config_{TARGET_TAG}_{SOURCE_TAG}.json"
sanity_path_repo = REPO_ROOT / "data/processed/supervised" / f"sanity_metrics_{TARGET_TAG}_{SOURCE_TAG}.json"
gaps_path_repo = REPO_ROOT / "data/processed/normalized_events" / "session_gap_thresholds.json"

dataloader_cfg = load_json(cfg_path_repo)
sanity_metrics = load_json(sanity_path_repo)
session_gaps = load_json(gaps_path_repo)

print("[09-01] Loaded dataloader_config keys:", list(dataloader_cfg.keys()))
print("[09-01] Loaded sanity_metrics keys:", list(sanity_metrics.keys()))
print("[09-01] Loaded session_gap_thresholds keys:", list(session_gaps.keys()))

# Enforce fixed decisions
assert session_gaps["target"]["primary_threshold_seconds"] == 1800, "Target gap must be 30m (1800s)."
assert session_gaps["source"]["primary_threshold_seconds"] == 600, "Source gap must be 10m (600s)."
print("[09-01] ✅ Session gaps confirmed: target=30m, source=10m")

proto = dataloader_cfg["protocol"]
K_LIST = [5, 10, 20]
MAX_K = max(K_LIST)

MAX_PREFIX_LEN = int(proto["max_prefix_len"])
CAP_ENABLED = bool(proto["source_long_session_policy"]["enabled"])
CAP_SESSION_LEN = int(proto["source_long_session_policy"]["cap_session_len"])
CAP_STRATEGY = str(proto["source_long_session_policy"]["cap_strategy"])

print("[09-01] Protocol from 06:")
print("  K_LIST:", K_LIST)
print("  MAX_PREFIX_LEN:", MAX_PREFIX_LEN)
print("  CAP_ENABLED:", CAP_ENABLED)
print("  CAP_SESSION_LEN:", CAP_SESSION_LEN)
print("  CAP_STRATEGY:", CAP_STRATEGY)

print("\n[09-01] CHECKPOINT A")
print("Confirm JSON loads + gap asserts passed.")


[09-01] REPO_ROOT: C:\mooc-coldstart-session-meta
[09-01] RUN_TAG: 20260102_231041
[09-01] Loaded dataloader_config keys: ['target', 'source', 'protocol']
[09-01] Loaded sanity_metrics keys: ['run_tag_target', 'run_tag_source', 'created_at', 'target', 'source', 'notes']
[09-01] Loaded session_gap_thresholds keys: ['generated_from_run_tag', 'generated_at', 'target', 'source', 'decision_notes']
[09-01] ✅ Session gaps confirmed: target=30m, source=10m
[09-01] Protocol from 06:
  K_LIST: [5, 10, 20]
  MAX_PREFIX_LEN: 20
  CAP_ENABLED: True
  CAP_SESSION_LEN: 200
  CAP_STRATEGY: take_last

[09-01] CHECKPOINT A
Confirm JSON loads + gap asserts passed.


Resolve artifact paths (target tensors + source sequences)

In [3]:
# [CELL 09-02] Resolve artifact paths (target tensors + source sequences) + existence checks

def must_exist(p: Path, label: str):
    if not p.exists():
        raise FileNotFoundError(f"{label} not found: {p}")
    return p

# TARGET tensors (05B output)
TARGET_TENSOR_DIR = REPO_ROOT / "data/processed/tensor_target"
target_train_pt = TARGET_TENSOR_DIR / f"target_tensor_train_{TARGET_TAG}.pt"
target_val_pt   = TARGET_TENSOR_DIR / f"target_tensor_val_{TARGET_TAG}.pt"
target_test_pt  = TARGET_TENSOR_DIR / f"target_tensor_test_{TARGET_TAG}.pt"
target_vocab_json = TARGET_TENSOR_DIR / f"target_vocab_items_{TARGET_TAG}.json"

# SOURCE sequences (05C output)
SOURCE_SEQ_ROOT = REPO_ROOT / "data/processed/session_sequences" / f"source_sessions_{SOURCE_TAG}"
source_train_dir = SOURCE_SEQ_ROOT / "train"
source_val_dir   = SOURCE_SEQ_ROOT / "val"
source_test_dir  = SOURCE_SEQ_ROOT / "test"
source_vocab_json = SOURCE_SEQ_ROOT / f"source_vocab_items_{SOURCE_TAG}.json"

for p, lbl in [
    (target_train_pt, "target_train_pt"),
    (target_val_pt, "target_val_pt"),
    (target_test_pt, "target_test_pt"),
    (target_vocab_json, "target_vocab_json"),
    (source_train_dir, "source_train_dir"),
    (source_val_dir, "source_val_dir"),
    (source_test_dir, "source_test_dir"),
    (source_vocab_json, "source_vocab_json"),
]:
    must_exist(p, lbl)

print("[09-02] ✅ All required artifacts exist")

print("\n[09-02] CHECKPOINT B")
print("If any artifact missing, STOP and paste the error.")


[09-02] ✅ All required artifacts exist

[09-02] CHECKPOINT B
If any artifact missing, STOP and paste the error.


Torch loader (PyTorch 2.6+) + vocab sizes + PAD/UNK + source token->id mapper

In [4]:
# [CELL 09-03] Torch loader (PyTorch 2.6+) + vocab sizes + PAD/UNK + source token->id mapper

def torch_load_repo_artifact(path, map_location="cpu"):
    path = str(path)
    try:
        obj = torch.load(path, map_location=map_location, weights_only=False)
        print(f"[09-03] torch.load OK (weights_only=False): {path}")
        return obj
    except TypeError:
        obj = torch.load(path, map_location=map_location)
        print(f"[09-03] torch.load OK (no weights_only arg): {path}")
        return obj

target_vocab = load_json(target_vocab_json)
source_vocab = load_json(source_vocab_json)

def infer_vocab_size(vocab: dict, name: str) -> int:
    for k in ["vocab_size", "n_items", "num_items", "size"]:
        if k in vocab:
            vs = int(vocab[k])
            print(f"[09-03] {name}: vocab_size from key '{k}' = {vs}")
            return vs
    if "vocab" in vocab and isinstance(vocab["vocab"], dict):
        d = vocab["vocab"]
        if len(d) == 0:
            return 0
        sample_k = next(iter(d.keys()))
        sample_v = d[sample_k]
        if isinstance(sample_v, int):
            ids = list(d.values())
            vs = max(ids) + 1 if len(ids) else 0
            print(f"[09-03] {name}: vocab_size from max(vocab values)+1 (token->id) = {vs}")
            return vs
    if "item2id" in vocab and isinstance(vocab["item2id"], dict):
        ids = list(vocab["item2id"].values())
        vs = max(ids) + 1 if len(ids) else 0
        print(f"[09-03] {name}: vocab_size from max(item2id values)+1 = {vs}")
        return vs
    raise KeyError(f"[09-03] {name}: Could not infer vocab_size. Keys={list(vocab.keys())}")

vocab_size_target = infer_vocab_size(target_vocab, "TARGET")
vocab_size_source = infer_vocab_size(source_vocab, "SOURCE")

def get_special_id(vocab_obj: dict, token_key: str, fallback: int, name: str) -> int:
    tok = vocab_obj.get(token_key, None)
    if tok is None:
        return fallback
    mapping = vocab_obj.get("vocab", {})
    if isinstance(mapping, dict) and tok in mapping and isinstance(mapping[tok], int):
        return int(mapping[tok])
    return fallback

PAD_ID_TARGET = get_special_id(target_vocab, "pad_token", 0, "TARGET")
UNK_ID_TARGET = get_special_id(target_vocab, "unk_token", 1, "TARGET")

PAD_ID_SOURCE = int(source_vocab.get("pad_id", 0))
UNK_ID_SOURCE = int(source_vocab.get("unk_id", 1))

print("[09-03] PAD_ID_TARGET:", PAD_ID_TARGET, "| UNK_ID_TARGET:", UNK_ID_TARGET)
print("[09-03] PAD_ID_SOURCE:", PAD_ID_SOURCE, "| UNK_ID_SOURCE:", UNK_ID_SOURCE)

assert PAD_ID_TARGET == 0
assert PAD_ID_SOURCE == 0

def build_token_to_id(vocab_obj: dict) -> dict:
    if "item2id" in vocab_obj and isinstance(vocab_obj["item2id"], dict):
        return vocab_obj["item2id"]
    if "vocab" in vocab_obj and isinstance(vocab_obj["vocab"], dict):
        d = vocab_obj["vocab"]
        if len(d) > 0 and isinstance(next(iter(d.values())), int):
            return d
    if "items" in vocab_obj and isinstance(vocab_obj["items"], list):
        return {tok: i for i, tok in enumerate(vocab_obj["items"])}
    raise KeyError(f"[09-03] Could not build token_to_id. Keys={list(vocab_obj.keys())}")

source_token_to_id = build_token_to_id(source_vocab)

def map_source_seq_to_ids(seq) -> np.ndarray:
    if seq is None:
        return np.array([], dtype=np.int64)
    if isinstance(seq, np.ndarray):
        seq_list = seq.tolist()
    else:
        seq_list = list(seq)
    if len(seq_list) == 0:
        return np.array([], dtype=np.int64)
    if isinstance(seq_list[0], (int, np.integer)):
        return np.asarray(seq_list, dtype=np.int64)
    return np.fromiter((source_token_to_id.get(tok, UNK_ID_SOURCE) for tok in seq_list), dtype=np.int64)

print("[09-03] ✅ Vocab + mapping ready")

print("\n[09-03] CHECKPOINT C")
print("Confirm vocab sizes + PAD/UNK before training.")


[09-03] TARGET: vocab_size from max(vocab values)+1 (token->id) = 747
[09-03] SOURCE: vocab_size from key 'vocab_size' = 1620
[09-03] PAD_ID_TARGET: 0 | UNK_ID_TARGET: 1
[09-03] PAD_ID_SOURCE: 0 | UNK_ID_SOURCE: 1
[09-03] ✅ Vocab + mapping ready

[09-03] CHECKPOINT C
Confirm vocab sizes + PAD/UNK before training.


Metrics (same as 06): HR/MRR/NDCG @ K={5,10,20}

In [5]:
# [CELL 09-04] Metrics (same as 06): HR/MRR/NDCG @ K={5,10,20}

def init_metrics():
    return {f"{m}@{k}": 0.0 for m in ["HR", "MRR", "NDCG"] for k in K_LIST}

def update_metrics_from_rank(metrics: dict, rank0: int | None):
    if rank0 is None:
        return
    r = rank0 + 1
    for k in K_LIST:
        if r <= k:
            metrics[f"HR@{k}"] += 1.0
            metrics[f"MRR@{k}"] += 1.0 / r
            metrics[f"NDCG@{k}"] += 1.0 / math.log2(r + 1.0)

def finalize_metrics(metrics: dict, n: int) -> dict:
    return {k: (float(v / n) if n > 0 else 0.0) for k, v in metrics.items()}

print("[09-04] ✅ Metric functions ready")


[09-04] ✅ Metric functions ready


GRU4Rec model definition (simple, CPU-friendly)

In [8]:
# [CELL 09-05] GRU4Rec model definition (simple, CPU-friendly)

class GRU4Rec(nn.Module):
    def __init__(self, vocab_size: int, emb_dim: int = 64, hidden_dim: int = 128, pad_id: int = 0):
        super().__init__()
        self.vocab_size = vocab_size
        self.pad_id = pad_id
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        self.gru = nn.GRU(input_size=emb_dim, hidden_size=hidden_dim, batch_first=True)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_ids: torch.Tensor, lengths: torch.Tensor):
        # input_ids: [B, T], lengths: [B]
        x = self.emb(input_ids)  # [B, T, D]
        packed = nn.utils.rnn.pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, h = self.gru(packed)  # h: [1, B, H]
        logits = self.out(h.squeeze(0))   # [B, V]
        return logits

print("[09-05] ✅ GRU4Rec defined")


[09-05] ✅ GRU4Rec defined


TARGET data tensors for training (from target_tensor_train_{TAG}.pt)
- We train on (input_ids, attn_mask -> lengths, labels).
- PAD excluded in loss using ignore_index=PAD_ID_TARGET.

In [9]:
# [CELL 09-06] TARGET data tensors for training (from target_tensor_train_{TAG}.pt)
# We train on (input_ids, attn_mask -> lengths, labels).
# PAD excluded in loss using ignore_index=PAD_ID_TARGET.

train_obj = torch_load_repo_artifact(target_train_pt, map_location="cpu")
val_obj   = torch_load_repo_artifact(target_val_pt, map_location="cpu")
test_obj  = torch_load_repo_artifact(target_test_pt, map_location="cpu")

def as_tensor_dict(obj: dict):
    return {
        "input_ids": torch.as_tensor(obj["input_ids"]).long(),
        "attn_mask": torch.as_tensor(obj["attn_mask"]).long(),
        "labels": torch.as_tensor(obj["labels"]).long(),
    }

target_train = as_tensor_dict(train_obj)
target_val   = as_tensor_dict(val_obj)
target_test  = as_tensor_dict(test_obj)

print("[09-06] TARGET train shapes:",
      tuple(target_train["input_ids"].shape),
      tuple(target_train["attn_mask"].shape),
      tuple(target_train["labels"].shape))

print("[09-06] TARGET val shapes:",
      tuple(target_val["input_ids"].shape),
      tuple(target_val["labels"].shape))

print("[09-06] TARGET test shapes:",
      tuple(target_test["input_ids"].shape),
      tuple(target_test["labels"].shape))

print("\n[09-06] CHECKPOINT D")
print("Confirm shapes match expectations (train rows=1944, seq_len=20).")


[09-03] torch.load OK (weights_only=False): C:\mooc-coldstart-session-meta\data\processed\tensor_target\target_tensor_train_20251229_163357.pt
[09-03] torch.load OK (weights_only=False): C:\mooc-coldstart-session-meta\data\processed\tensor_target\target_tensor_val_20251229_163357.pt
[09-03] torch.load OK (weights_only=False): C:\mooc-coldstart-session-meta\data\processed\tensor_target\target_tensor_test_20251229_163357.pt
[09-06] TARGET train shapes: (1944, 20) (1944, 20) (1944,)
[09-06] TARGET val shapes: (189, 20) (189,)
[09-06] TARGET test shapes: (200, 20) (200,)

[09-06] CHECKPOINT D
Confirm shapes match expectations (train rows=1944, seq_len=20).


TARGET training loop (small dataset, CPU OK)
- Logs training loss and evaluates on VAL each epoch.
- NOTE: This is target-only GRU4Rec baseline (Layer-1).

In [10]:
# [CELL 09-07] TARGET training loop (small dataset, CPU OK)
# Logs training loss and evaluates on VAL each epoch.
# NOTE: This is target-only GRU4Rec baseline (Layer-1).

GRU_CFG = {
    "emb_dim": 64,
    "hidden_dim": 128,
    "batch_size": 256,
    "epochs": 10,
    "lr": 1e-3,
    "weight_decay": 0.0,
    "grad_clip": 1.0,
    "seed": 42,
}

print("[09-07] GRU_CFG:", GRU_CFG)

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(GRU_CFG["seed"])

device = torch.device("cpu")

model_t = GRU4Rec(vocab_size=vocab_size_target,
                 emb_dim=GRU_CFG["emb_dim"],
                 hidden_dim=GRU_CFG["hidden_dim"],
                 pad_id=PAD_ID_TARGET).to(device)

opt = torch.optim.Adam(model_t.parameters(), lr=GRU_CFG["lr"], weight_decay=GRU_CFG["weight_decay"])

def make_lengths(attn_mask: torch.Tensor) -> torch.Tensor:
    return attn_mask.sum(dim=1).long()

def iter_batches(data: dict, batch_size: int, shuffle: bool = True):
    n = data["input_ids"].shape[0]
    idx = np.arange(n)
    if shuffle:
        np.random.shuffle(idx)
    for s in range(0, n, batch_size):
        b = idx[s:s+batch_size]
        yield (
            data["input_ids"][b].to(device),
            make_lengths(data["attn_mask"][b]).to(device),
            data["labels"][b].to(device),
        )

def eval_gru(model: GRU4Rec, data: dict, split_name: str) -> dict:
    model.eval()
    metrics = init_metrics()
    n = 0
    with torch.no_grad():
        for x, lengths, y in iter_batches(data, batch_size=GRU_CFG["batch_size"], shuffle=False):
            logits = model(x, lengths)  # [B, V]
            # mask PAD from ranking by setting it to -inf
            logits[:, PAD_ID_TARGET] = -1e9
            topk = torch.topk(logits, k=MAX_K, dim=1).indices.cpu().numpy()  # [B, K]
            y_np = y.cpu().numpy()
            for i in range(topk.shape[0]):
                if int(y_np[i]) == PAD_ID_TARGET:
                    continue
                row = topk[i]
                # rank lookup
                pos = np.where(row == int(y_np[i]))[0]
                rank0 = int(pos[0]) if pos.size > 0 else None
                update_metrics_from_rank(metrics, rank0)
                n += 1
    out = finalize_metrics(metrics, n)
    out["_n_examples"] = int(n)
    return out

# Train
train_losses = []
for epoch in range(1, GRU_CFG["epochs"] + 1):
    model_t.train()
    t0 = time.time()
    total_loss = 0.0
    total_n = 0

    for x, lengths, y in iter_batches(target_train, GRU_CFG["batch_size"], shuffle=True):
        opt.zero_grad()
        logits = model_t(x, lengths)  # [B, V]
        loss = F.cross_entropy(logits, y, ignore_index=PAD_ID_TARGET)
        loss.backward()
        nn.utils.clip_grad_norm_(model_t.parameters(), GRU_CFG["grad_clip"])
        opt.step()

        bs = x.shape[0]
        total_loss += float(loss.item()) * bs
        total_n += bs

    dt = time.time() - t0
    avg_loss = total_loss / max(1, total_n)
    train_losses.append(avg_loss)
    val_metrics = eval_gru(model_t, target_val, "target_val")

    print(f"[09-07] epoch={epoch:02d}/{GRU_CFG['epochs']} loss={avg_loss:.4f} time={dt:.1f}s | VAL:", val_metrics)

print("\n[09-07] CHECKPOINT E")
print("Paste the epoch logs (loss + VAL metrics). If VAL gets worse, we can reduce epochs or adjust hidden_dim.")


[09-07] GRU_CFG: {'emb_dim': 64, 'hidden_dim': 128, 'batch_size': 256, 'epochs': 10, 'lr': 0.001, 'weight_decay': 0.0, 'grad_clip': 1.0, 'seed': 42}
[09-07] epoch=01/10 loss=6.6174 time=0.8s | VAL: {'HR@5': 0.005291005291005291, 'HR@10': 0.031746031746031744, 'HR@20': 0.037037037037037035, 'MRR@5': 0.0026455026455026454, 'MRR@10': 0.006508776350046192, 'MRR@20': 0.006839464180734022, 'NDCG@5': 0.003338252664399246, 'NDCG@10': 0.01222772904824418, 'NDCG@20': 0.013522176361039025, '_n_examples': 189}
[09-07] epoch=02/10 loss=6.5630 time=0.4s | VAL: {'HR@5': 0.031746031746031744, 'HR@10': 0.037037037037037035, 'HR@20': 0.0582010582010582, 'MRR@5': 0.009611992945326277, 'MRR@10': 0.010199882422104643, 'MRR@20': 0.011569319085658954, 'NDCG@5': 0.015001650402719228, 'NDCG@10': 0.01659440170252865, 'NDCG@20': 0.021820438050107506, '_n_examples': 189}
[09-07] epoch=03/10 loss=6.5069 time=0.3s | VAL: {'HR@5': 0.031746031746031744, 'HR@10': 0.047619047619047616, 'HR@20': 0.08465608465608465, 'MR

TARGET final evaluation on TEST

In [11]:
# [CELL 09-08] TARGET final evaluation on TEST

t_val_gru = eval_gru(model_t, target_val, "target_val")
t_test_gru = eval_gru(model_t, target_test, "target_test")

print("[09-08] TARGET VAL (GRU4Rec):", t_val_gru)
print("[09-08] TARGET TEST (GRU4Rec):", t_test_gru)

print("\n[09-08] CHECKPOINT F")
print("Paste TARGET VAL/TEST metrics before we write reports.")


[09-08] TARGET VAL (GRU4Rec): {'HR@5': 0.05291005291005291, 'HR@10': 0.1164021164021164, 'HR@20': 0.15873015873015872, 'MRR@5': 0.020194003527336864, 'MRR@10': 0.029251700680272108, 'MRR@20': 0.03209694209067642, 'NDCG@5': 0.028188332253080303, 'NDCG@10': 0.049299947817881065, 'NDCG@20': 0.05990017040185571, '_n_examples': 189}
[09-08] TARGET TEST (GRU4Rec): {'HR@5': 0.08, 'HR@10': 0.15, 'HR@20': 0.215, 'MRR@5': 0.04699999999999999, 'MRR@10': 0.056803571428571425, 'MRR@20': 0.06119113452337136, 'NDCG@5': 0.05513923995665121, 'NDCG@10': 0.07822673444977948, 'NDCG@20': 0.0945156893266747, '_n_examples': 200}

[09-08] CHECKPOINT F
Paste TARGET VAL/TEST metrics before we write reports.


Write report artifacts to reports/09_gru4rec_baseline/<RUN_TAG>/ + update meta.json
- Note: This notebook covers TARGET GRU4Rec baseline first (source training comes later when we decide feasibility).

In [12]:
# [CELL 09-09] Write report artifacts to reports/09_gru4rec_baseline/<RUN_TAG>/ + update meta.json
# Note: This notebook covers TARGET GRU4Rec baseline first (source training comes later when we decide feasibility).

REPORT_DIR = REPO_ROOT / "reports" / "09_gru4rec_baseline" / RUN_TAG
REPORT_DIR.mkdir(parents=True, exist_ok=True)

def save_json(obj: dict, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)

run_meta = {
    "run_tag": RUN_TAG,
    "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "inputs": {
        "target_run_tag": TARGET_TAG,
        "source_run_tag": SOURCE_TAG,
        "target_train_pt": str(target_train_pt),
        "target_val_pt": str(target_val_pt),
        "target_test_pt": str(target_test_pt),
        "target_vocab_json": str(target_vocab_json),
        "dataloader_config": str(cfg_path_repo),
        "sanity_metrics": str(sanity_path_repo),
        "session_gap_thresholds": str(gaps_path_repo),
    },
    "protocol_reused_from_06": {
        "K_LIST": K_LIST,
        "MAX_PREFIX_LEN": MAX_PREFIX_LEN,
        "PAD_ID_TARGET": PAD_ID_TARGET,
        "pad_excluded_from_ranking": True,
    },
    "model": {
        "name": "GRU4Rec",
        "vocab_size": int(vocab_size_target),
        "emb_dim": int(GRU_CFG["emb_dim"]),
        "hidden_dim": int(GRU_CFG["hidden_dim"]),
    },
    "train_cfg": GRU_CFG,
    "notes": [
        "This run trains/evaluates GRU4Rec on TARGET only (Layer-1 baseline).",
        "Source GRU4Rec training will require a streaming sampler over source sessions and is handled separately.",
    ],
}

results = {
    "target": {
        "val": t_val_gru,
        "test": t_test_gru,
        "train_losses": train_losses,
    },
    "source": None,
}

save_json(run_meta, REPORT_DIR / "run_meta.json")
save_json(results, REPORT_DIR / "results.json")

# Save model checkpoint (optional but useful)
ckpt = {
    "state_dict": model_t.state_dict(),
    "gru_cfg": GRU_CFG,
    "vocab_size_target": vocab_size_target,
    "pad_id": PAD_ID_TARGET,
}
torch.save(ckpt, REPORT_DIR / "model.pt")
print("[09-09] ✅ Saved model checkpoint:", REPORT_DIR / "model.pt")

# Update meta.json
meta_path = REPO_ROOT / "meta.json"
meta = load_json(meta_path) if meta_path.exists() else {"artifacts": {}}

meta.setdefault("artifacts", {})
meta["artifacts"].setdefault("gru4rec_baseline", {})
meta["artifacts"]["gru4rec_baseline"][RUN_TAG] = {
    "target_run_tag": TARGET_TAG,
    "source_run_tag": SOURCE_TAG,
    "report_dir": str(REPORT_DIR),
    "results_json": str(REPORT_DIR / "results.json"),
    "run_meta_json": str(REPORT_DIR / "run_meta.json"),
    "model_pt": str(REPORT_DIR / "model.pt"),
}
meta["updated_at"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
save_json(meta, meta_path)

print("[09-09] ✅ Wrote report files under:", REPORT_DIR)
print("[09-09] ✅ Updated meta.json:", meta_path)

print("\n[09-09] CHECKPOINT G")
print("Paste: report dir + confirm meta.json updated.")


[09-09] ✅ Saved model checkpoint: C:\mooc-coldstart-session-meta\reports\09_gru4rec_baseline\20260102_231041\model.pt
[09-09] ✅ Wrote report files under: C:\mooc-coldstart-session-meta\reports\09_gru4rec_baseline\20260102_231041
[09-09] ✅ Updated meta.json: C:\mooc-coldstart-session-meta\meta.json

[09-09] CHECKPOINT G
Paste: report dir + confirm meta.json updated.


Footer summary

In [13]:
# [CELL 09-10] Footer summary

print("========== 09 GRU4Rec Baseline Summary ==========")
print("RUN_TAG:", RUN_TAG)
print("--- TARGET ---")
print("VAL :", t_val_gru)
print("TEST:", t_test_gru)
print("Report dir:", REPORT_DIR)
print("================================================")


========== 09 GRU4Rec Baseline Summary ==========
RUN_TAG: 20260102_231041
--- TARGET ---
VAL : {'HR@5': 0.05291005291005291, 'HR@10': 0.1164021164021164, 'HR@20': 0.15873015873015872, 'MRR@5': 0.020194003527336864, 'MRR@10': 0.029251700680272108, 'MRR@20': 0.03209694209067642, 'NDCG@5': 0.028188332253080303, 'NDCG@10': 0.049299947817881065, 'NDCG@20': 0.05990017040185571, '_n_examples': 189}
TEST: {'HR@5': 0.08, 'HR@10': 0.15, 'HR@20': 0.215, 'MRR@5': 0.04699999999999999, 'MRR@10': 0.056803571428571425, 'MRR@20': 0.06119113452337136, 'NDCG@5': 0.05513923995665121, 'NDCG@10': 0.07822673444977948, 'NDCG@20': 0.0945156893266747, '_n_examples': 200}
Report dir: C:\mooc-coldstart-session-meta\reports\09_gru4rec_baseline\20260102_231041
